# 2 Data wrangling<a id='2_Data_wrangling'></a>

## 2.1 Contents<a id='2.1_Contents'></a>
* [2 Data wrangling](#2_Data_wrangling)
  * [2.1 Contents](#2.1_Contents)
  * [2.2 Introduction](#2.2_Introduction)
  * [2.3 Imports](#2.3_Imports)
  * [2.4 Objectives](#2.4_Objectives)
  * [2.5 Load The Perfume Data](#2.5_Load_The_Perfume_Data)
  * [2.6 Explore The Data](#2.6_Explore_The_Data)
    * [2.6.1 Create Profile Report for the dataset](#2.6.1_Create_profile_report)
    * [2.6.2 Number Of Missing Values By Column](#2.6.2_Number_Of_Missing_Values_By_Column)
    * [2.6.3 Categorical Features](#2.6.3_Categorical_Features)
      * [2.6.3.1 Unique Resort Names](#2.6.3.1_Unique_Resort_Names)
      * [2.6.3.2 Region And State](#2.6.3.2_Region_And_State)
      * [2.6.3.3 Number of distinct regions and states](#2.6.3.3_Number_of_distinct_regions_and_states)
      * [2.6.3.4 Distribution Of Resorts By Region And State](#2.6.3.4_Distribution_Of_Resorts_By_Region_And_State)
      * [2.6.3.5 Distribution Of Ticket Price By State](#2.6.3.5_Distribution_Of_Ticket_Price_By_State)
        * [2.6.3.5.1 Average weekend and weekday price by state](#2.6.3.5.1_Average_weekend_and_weekday_price_by_state)
        * [2.6.3.5.2 Distribution of weekday and weekend price by state](#2.6.3.5.2_Distribution_of_weekday_and_weekend_price_by_state)
    * [2.6.4 Numeric Features](#2.6.4_Numeric_Features)
      * [2.6.4.1 Numeric data summary](#2.6.4.1_Numeric_data_summary)
      * [2.6.4.2 Distributions Of Feature Values](#2.6.4.2_Distributions_Of_Feature_Values)
        * [2.6.4.2.1 SkiableTerrain_ac](#2.6.4.2.1_SkiableTerrain_ac)
        * [2.6.4.2.2 Snow Making_ac](#2.6.4.2.2_Snow_Making_ac)
        * [2.6.4.2.3 fastEight](#2.6.4.2.3_fastEight)
        * [2.6.4.2.4 fastSixes and Trams](#2.6.4.2.4_fastSixes_and_Trams)
  * [2.7 Derive State-wide Summary Statistics For Our Market Segment](#2.7_Derive_State-wide_Summary_Statistics_For_Our_Market_Segment)
  * [2.8 Drop Rows With No Price Data](#2.8_Drop_Rows_With_No_Price_Data)
  * [2.9 Review distributions](#2.9_Review_distributions)
  * [2.10 Population data](#2.10_Population_data)
  * [2.11 Target Feature](#2.11_Target_Feature)
    * [2.11.1 Number Of Missing Values By Row - Resort](#2.11.1_Number_Of_Missing_Values_By_Row_-_Resort)
  * [2.12 Save data](#2.12_Save_data)
  * [2.13 Summary](#2.13_Summary)


## 2.2 Introduction<a id='2.2_Introduction'></a>

The purpose of this data science project is to classify a given perfume to a category of gender and age that would receive it the best.

This will help the New Sense Company to market their perfume to the right crowd.

## 2.3 Imports<a id='2.3_Imports'></a>

The autoreload instruction reloads modules automatically before code execution, which is helpful for the update below.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# To print all outputs in a cell without using a print() funciton

#from IPython.core.interactiveshell import InteractiveShell  
#InteractiveShell.ast_node_interactivity = "all"

# To print only the last statement
#InteractiveShell.ast_node_interactivity = "last_expr"

# Enable Widgets in notebook
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [3]:
#importing ds packages
import pandas as pd
import numpy as np

import pandas_profiling
#from pandas_profiling.utils.cache import cache_file

In [4]:
# Visualization packages

# # To make charts show up in Jupyter notebook itself.
# %matplotlib inline 

# # This will give charts inline with interactive controls.
# #%matplotlib notebook 

import matplotlib.pyplot as plt
import seaborn as sns

# sns.set_style('whitegrid')

In [5]:
# System OS 
# import os

## 2.4 Objectives<a id='2.4_Objectives'></a>

There are some fundamental questions to resolve in this notebook before you move on.

* Do you think you may have the data you need to tackle the desired question?
    * Have you identified the required target value?
    * Do you have potentially useful features?
* Do you have any fundamental issues with the data?

## 2.5 Load The Perfume Data<a id='2.5_Load_The_Perfume_Data'></a>

In [6]:
# importing the data
df = pd.read_csv('../data/input/perfume.csv')#, nrows=5000)

In [7]:
df.shape

(51212, 86)

## 2.6 Explore The Data<a id='2.6_Explore_The_Data'></a>

### 2.6.1 Create Profile report<a id='2.6.1_Create_profile_report'></a>

In [ ]:
# Full Report
profile_report_full = df.profile_report(sort='None', html={'style':{'full_width': True}}, progress_bar=False, config_file='config_limited.yaml')

profile_report_full.to_widgets()